### Brief Prompt and Strategy

Eurovision voting history dataset: https://www.kaggle.com/datagraver/eurovision-song-contest-scores-19752017   

Based on the history of the voting for Eurovision, can we predict the next winners? The dataset above contains the votes from 1975 to 2017. Therefore, I can test my model for 2018 and then predict the 2019's winners (this weekend!).

While the voting should either be random or reflective of true talent/ common crowd perception, it is known that there exist Eurovision voting schemes. Therefore I will attempt to use the history of voting to forecast.

My approach contains the following:
- use only the final votes for a consistent dataset across the years
- predict an individual country's votes, then loop over all countries to identify the winner
- do so using a two step approach: 1) using history of their votes and who voted for them, identify the countries that a given country would vote for via logistic regression. 2) then apply linear regression to assign the number of votes to those countries.

### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Read in the data

In [2]:
orig_data=pd.read_csv("votes.csv")
orig_data.head()

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate
0,1975,f,1975f,J,Belgium,Belgium,0,x
1,1975,f,1975f,J,Belgium,Finland,0,NaN
2,1975,f,1975f,J,Belgium,France,2,NaN
3,1975,f,1975f,J,Belgium,Germany,0,NaN
4,1975,f,1975f,J,Belgium,Ireland,12,NaN


### Clean the data a bit

In [3]:
#correct incorrect spelling
orig_data.replace(to_replace=["The Netherands"], value=["The Netherlands"], inplace=True)

In [8]:
#lets drop the duplicate lines from the table (that is when To = From)
orig_data.drop(orig_data[orig_data["Duplicate"] == "x"].index, inplace=True)

#now lets get rid of that column
orig_data.drop(columns="Duplicate", inplace=True)

In [4]:
orig_data.rename(columns={"(semi-) final":"Final", "Jury or Televoting":"Method","Points      ":"Points"}, inplace=True)

In [9]:
orig_data.head()

,Year,Final,Edition,Method,From country,To country,Points
1,1975,f,1975f,J,Belgium,Finland,0
2,1975,f,1975f,J,Belgium,France,2
3,1975,f,1975f,J,Belgium,Germany,0
4,1975,f,1975f,J,Belgium,Ireland,12
5,1975,f,1975f,J,Belgium,Israel,1


First we note that voting changed, and therefore info changed over the years

In [18]:
orig_data["Final"].unique()

array(['f', 'sf', 'sf1', 'sf2'], dtype=object)

In [19]:
orig_data[orig_data["Final"]=="f"]["Year"].unique()

array([1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

Keep only the finals data. For now, both jury and televote.

In [12]:
finals=orig_data[orig_data["Final"]=="f"].copy()

In [23]:
finals.head()

,Year,Final,Edition,Method,From country,To country,Points
1,1975,f,1975f,J,Belgium,Finland,0
2,1975,f,1975f,J,Belgium,France,2
3,1975,f,1975f,J,Belgium,Germany,0
4,1975,f,1975f,J,Belgium,Ireland,12
5,1975,f,1975f,J,Belgium,Israel,1


In [36]:
#for now, only inclue Jury voting
finals=finals[finals["Method"]=="J"]

### Set up a table given a country.
row =  year

columns = to and from votes

In [37]:
finals[finals["Year"] == 1990].head()

,Year,Final,Edition,Method,From country,To country,Points
5787,1990,f,1990f,J,Austria,Belgium,4
5788,1990,f,1990f,J,Austria,Cyprus,0
5789,1990,f,1990f,J,Austria,Denmark,6
5790,1990,f,1990f,J,Austria,Finland,0
5791,1990,f,1990f,J,Austria,France,2


In [64]:
# all the votes from the choice
given=finals[finals["From country"] == "Austria"]
given.drop(columns=["Final","Edition","Method","From country"],inplace=True)
given=given.pivot(index="To country",columns="Year",values="Points")

In [65]:
# all of the votes to the choice
recieved=finals[finals["To country"] == "Austria"]
recieved.drop(columns=["Final","Edition","Method","To country"],inplace=True)
recieved=recieved.pivot(index="From country",columns="Year",values="Points")

In [66]:
#save the transpose to get the desired format
recieved=recieved.T
given=given.T

In [67]:
#rename the country columns so we can combine the two tables
recieved.rename(columns = lambda x: "Recieved from: "+x, inplace=True)
given.rename(columns = lambda x: "Given to: "+x, inplace=True)

In [72]:
#join the datasets
country_table=given.join(recieved, on="Year")

In [107]:
country_table.fillna(value=0, inplace=True)

### predict voting for a given country

In [177]:
#keep 2018 as my one test
country_table_train=country_table.drop(index=2018)

In [178]:
#label for a one countries votes

#the label needs to be for the year after, so you give it 2016 and it predicts for 2017.
#what happens each year
label_albania=country_table_train["Given to: Albania"] > 0
#shift the result to the next year, so each row of data forms a prediction
labels_shifted=label_albania.rename(index = lambda x: 1+x)
#cheat by copying the first year
label_albania=label_albania[[label_albania.index.values[0]]].append(labels_shifted)
#drop the final year so it matches the correct length
label_albania.drop(2018,inplace=True)

In [179]:
#make 1 or 0 instead of Boolean
label_albania=label_albania.astype(int)

In [180]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

In [181]:
logmodel.fit(country_table_train, label_albania)

/anaconda3/envs/myproject/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [182]:
logmodel.predict

<bound method LinearClassifierMixin.predict of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

In [183]:
country_table_train

,Given to: Albania,Given to: Armenia,Given to: Australia,Given to: Austria,Given to: Azerbaijan,Given to: Belarus,Given to: Belgium,Given to: Bosnia & Herzegovina,Given to: Bulgaria,Given to: Croatia,...,Recieved from: Slovakia,Recieved from: Slovenia,Recieved from: Spain,Recieved from: Sweden,Recieved from: Switzerland,Recieved from: The Netherlands,Recieved from: Turkey,Recieved from: Ukraine,Recieved from: United Kingdom,Recieved from: Yugoslavia
Year,,,,,,,,,,,,,,,,,,,,,
1976,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,3.0,7.0,0.0,0.0,4.0,2.0
1977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1978,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0
1979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,1.0,4.0,3.0,0.0,0.0,6.0,0.0
1981,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,6.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0
1982,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,7.0,0.0,10.0,4.0
1983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,4.0,10.0,0.0,3.0,4.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
a=country_table_train["Given to: Albania"] > 0
a.rename(index = lambda x: 1+x, inplace=True)

In [149]:
a=list(a.astype(int))
[a[0]]+a

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [176]:
#what happens each year
label_albania=country_table_train["Given to: Albania"] > 0
#shift the result to the next year, so each row of data forms a prediction
labels_shifted=label_albania.rename(index = lambda x: 1+x)
label_albania=label_albania[[label_albania.index.values[0]]].append(labels_shifted)
#drop the final year so it matches the correct length
label_albania.drop(2018,inplace=True)

In [171]:
label_albania.index.values[0]

1976